- https://www.salabim.org/manual/


<img src="../images/Enexis_JADS.png" alt="Enexis JADS" style="width:12%; float:right">

# EVSE charging facility


### Initialization

In [ ]:
# arrival rate of EVs and energy requirement rate
ev_arrival_rate = 40 # EV per hour
energy_req_rate = 50 # KW per hour

# Create a list with EVSE's
sim_evse = range(1, 10)
sim_time = 5000  # minutes
sim_reps = 20  # simulations

#### Python SetUp

In [ ]:
# Setup file for the MMC simulation
# load packages
import salabim as sim
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from datetime import datetime
from scipy.special import factorial

# https://brandfetch.com/enexis.nl?view=library&library=default&collection=colors

ENEXIS_G_0 = "#B6D300"
ENEXIS_G_1 = "#BDDB00"
ENEXIS_G_2 = "#A3C100"
ENEXIS_P_0 = "#DA1369"
ENEXIS_P_1 = "#EE6DAE"
ENEXIS_P_2 = "#DF0073"
ENEXIS_B_0 = "#04296C"

# Create a string with the simulation session name
sim_session_name = (
    "R4_"
    + "df_EVSE_"
    + str(sim_evse[0])
    + "-"
    + str(sim_evse[-1])
    + "_time_"
    + str(sim_time)
    + "_reps_"
    + str(sim_reps)
    + ".csv"
)

ffn_results = "./sim_results/" + sim_session_name

print(f"results will be saved here: {ffn_results}")

### Simulation Model

In [ ]:
# simulation code for MMC queue
# function to run simulation
def sim_facility(
    ev_arrival_rate,  # EV's per hour
    energy_req_rate,  # KW per hour
    number_of_EVSE=1,  # number of EVSE's
    sim_time=50000,
    time_unit="minutes",
    random_seed=42,
    run=1,
):
    # Generator which creates EV's
    class EV_Generator(sim.Component):
        # setup method is called when the component is created
        # and is used to initialize the component
        # switch off monitoring for mode and status
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

        def process(self):
            while True:
                EV()
                self.hold(inter_arr_time_distr.sample())

    class EV(sim.Component):
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

        def process(self):
            self.enter(waitingline)
            for EVSE in facility:
                if EVSE.ispassive():
                    EVSE.activate()
                    break  # activate at most one charging station
            self.passivate()

    class EVSE(sim.Component):
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

            self.length = sim.Monitor(
                name="length", monitor=True, level=True, type="int32"
            )
            self.length_of_stay = sim.Monitor(
                name="length_of_stay", monitor=True, level=False, type="float"
            )
            self.power_mon = sim.Monitor(
                name="power.", monitor=True, level=True, type="float"
            )
            self.power = 1.0

        def process(self):
            while True:
                self.length.tally(0)
                while len(waitingline) == 0:
                    self.set_mode("Waiting")
                    self.passivate()
                self.car = waitingline.pop()
                self.length.tally(1)
                # wf = app.now()
                energy_request = energy_request_distr.sample()
                charging_time = energy_request / self.power
                # monEVSE_IDLE.tally(wf - ws)
                self.length_of_stay.tally(charging_time)
                self.power_mon.tally(self.power)
                self.set_mode("Charging")
                self.hold(charging_time)
                self.car.activate()

    # Create distributions
    inter_arr_time_distr = sim.Exponential(60 / ev_arrival_rate)  # minutes between EV's
    energy_request_distr = sim.Exponential(60 / energy_req_rate)  # minutes to charge EV

    # https://www.salabim.org/manual/Reference.html#environment
    app = sim.App(
        trace=False,  # defines whether to trace or not
        random_seed=random_seed,  # if “*”, a purely random value (based on the current time)
        time_unit=time_unit,  # defines the time unit used in the simulation
        name="Charging Station",  # name of the simulation
        do_reset=True,  # defines whether to reset the simulation when the run method is called
        yieldless=True,  # defines whether the simulation is yieldless or not
    )

    # Instantiate and activate the client generator
    EV_Generator(name="Electric Vehicles Generator")

    # Create Queue and set monitor to stats_only
    # https://www.salabim.org/manual/Queue.html
    waitingline = sim.Queue(name="Waiting EV's", monitor=True)
    # waitingline.length_of_stay.monitor(value=True)
    waitingline.length.reset_monitors(stats_only=True)
    waitingline.length_of_stay.reset_monitors(stats_only=True)

    # Instantiate the EVSE's, list comprehension
    facility = [EVSE() for _ in range(number_of_EVSE)]

    # Execute Simulation
    app.run(till=sim_time)

    # Calculate aggregate statistics
    total_evse_stay = sum(x.length_of_stay for x in facility).mean()
    total_evse_lngt = sum(x.length for x in facility).mean()

    # waitingline.mode.print_histogram(values=True)

    # Return results
    return {
        "run": run,
        "lambda": ev_arrival_rate,
        "mu": energy_req_rate,
        "c": number_of_EVSE,
        "RO": total_evse_lngt / number_of_EVSE,
        "P0": 0,
        "Lq": waitingline.length.mean(),
        "Wq": waitingline.length_of_stay.mean(),
        "Ls": total_evse_lngt + waitingline.length.mean(),
        "Ws": total_evse_stay + waitingline.length_of_stay.mean(),
    }


# function to run simulation X times
# returns a DataFrame with the results
def sim_x_facility(
    ev_arrival_rate,
    energy_req_rate,
    number_of_EVSE=1,
    sim_time=50000,
    number_of_simulations=30,
    verbose=False,
):
    # Create empty list to store results
    sim_runs = []

    # Run simulation X times with different random seeds for same number of EVSE's
    for i in range(number_of_simulations):
        sim_runs.append(
            sim_facility(
                ev_arrival_rate=ev_arrival_rate,
                energy_req_rate=energy_req_rate,
                number_of_EVSE=number_of_EVSE,
                sim_time=sim_time,
                random_seed=i,
                run=i,
            )
        )
        if verbose:
            print(f"EVSE's {number_of_EVSE}, run {i} completed at {datetime.now()}")

    # Concatenate all runs
    return pd.DataFrame(sim_runs )


# function to run simulation X times per EVSE for all EVSE's
def sim_x_facility_per_evse(
    ev_arrival_rate,
    energy_req_rate,
    number_of_EVSE=1,
    sim_time=50000,
    number_of_simulations=30,
    fixed_utilization=True,
    verbose=False,
):
    # Initialize an empty list
    dfs = []

    # Execute the simulation for each EVSE
    for j in number_of_EVSE:
        # Execute simulation
        df = sim_x_facility(
            ev_arrival_rate=j * ev_arrival_rate if fixed_utilization else ev_arrival_rate,
            energy_req_rate=energy_req_rate,
            number_of_EVSE=j,
            sim_time=sim_time,
            number_of_simulations=number_of_simulations,
            verbose=verbose,
        )
        # Append df to dfs
        dfs.append(df)

    # Concatenate all DataFrames in dfs
    return pd.concat(dfs, axis =0, ignore_index=True)

## Run Simulation

In [ ]:
# %%script false --no-raise-error # comment out this line to run the simulation CTRL + /
df_total = sim_x_facility_per_evse(
    ev_arrival_rate=ev_arrival_rate,
    energy_req_rate=energy_req_rate,
    number_of_EVSE=sim_evse,
    sim_time=sim_time,
    number_of_simulations=sim_reps,
    verbose=True,
)

# print results of simulation
print(df_total)

# save results of simulation
df_total.to_csv(
    path_or_buf=ffn_results,
    sep=";",
    index=False,
    header=True,
    decimal=".",
    float_format="%.3f",
)

### Analysis

For each performance indicator we will determine the mena and a confidence interval

In [ ]:
# student-t distribution
# https://en.wikipedia.org/wiki/Student%27s_t-distribution#Table_of_selected_values
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html
def t_sd(df, col, confidence_interval, sides="both", decimals=3):
    if sides != "both":
        ci = confidence_interval
    else:
        ci = confidence_interval + (1 - confidence_interval) / 2

    # column of interest
    x = df[col]

    # number of SD for confidence interval 
    # Note: ddof=1 for sample
    tdst = st.t.ppf(ci, df=len(x) - 1)

    return pd.DataFrame(
        {
            "c": df["c"].unique()[0],
            "name": col,
            "mean": x.mean().round(decimals),
            "lbnd": (x.mean() - tdst * x.std(ddof=1) / np.sqrt(len(x))).round(decimals),
            "ubnd": (x.mean() + tdst * x.std(ddof=1) / np.sqrt(len(x))).round(decimals),
            "stdv": x.std(ddof=1).round(decimals),
            "tdst": tdst.round(decimals),
            "runs": len(x),
        },
        index=[0],
    )


# Calculate mean and confidence interval for waiting time
# df_evse = df_sim.groupby('c')['Wq'].agg(['mean', 'count']).reset_index()

# def mean_confidence_interval(data, confidence=0.95):
#     a = 1.0 * np.array(data)
#     n = len(a)
#     # m, se = np.mean(a), scipy.stats.sem(a)
#     m, se = np.mean(a), st.t.interval(confidence, n - 1, loc=np.mean(a), scale=st.sem(a))
#     return m, se


def sim_mean_and_ci(df_sim, col):
    # Calculate the mean and count for each 'c' group

    res = []
    for c in df_sim["c"].unique():
        df_evse = df_sim[df_sim["c"] == c]

        res.append(
            t_sd(df_evse, col, confidence_interval=0.95, sides="both", decimals=2)
        )

    # return results
    return pd.concat(res, axis=0, ignore_index=True)

In [ ]:
# read df_total from disk csv
df_sim = pd.read_csv(ffn_results, sep=";", decimal=".", index_col=False)

# Create an empty DataFrame to store the results
res = []

# Loop over each column in the DataFrame
for column in ['lambda', 'mu', 'RO', 'P0', 'Lq', 'Wq', 'Ls', 'Ws']:
    # Apply the function to the column and store the result
    res.append(sim_mean_and_ci(df_sim, df_sim[column].name))

df_res = pd.concat(res, axis=0, ignore_index=True)
print(df_res)

In [ ]:
df_pivot = (
    df_res.pivot(columns="name", values="mean", index="c")
    .assign(runs=sim_reps)
    [["lambda", "mu", "RO", "P0", "Lq", "Wq", "Ls", "Ws", "runs" ]]
)

print(df_pivot)

#### under development

In [ ]:
# # import matplotlib.pyplot as plt

# # plt.plot(*waitingline.length.tx(), linewidth=1, color="red")
# # plt.show()

# # Print server statistics
# I = monEVSE_IDLE.mean()
# P = monEVSE_PROC.mean()
# R = (P * monEVSE_PROC.number_of_entries()) / (
#     (I * monEVSE_IDLE.number_of_entries()) + (P * monEVSE_PROC.number_of_entries())
# )

# print(f"IDLE: {I}, PROC: {P}, UTIL: {R}")

# def prtEVSE(evse):
#     print(
#         f"name: {evse.proc.name()}, seq: {evse.proc.sequence_number()}, mean: {evse.proc.mean()}"
#     )


# [prtEVSE(x) in ChargingStations for x in ChargingStations]

# print(f"Total: {sum(x.proc for x in ChargingStations).mean()}")
# ChargingStations[0].status.print_histogram(values=True)
# ChargingStations[0].mode.print_histogram(values=True)

# System SetUp

In [31]:
import sys
import platform
from importlib_metadata import version

print(f"python implementation: {platform.python_implementation()}")
print(f"python version: {sys.version}")
print(f"salabim: {version('salabim')}")

python implementation: CPython
python version: 3.9.18 | packaged by conda-forge | (main, Aug 30 2023, 03:49:32) 
[GCC 12.3.0]
salabim: 23.3.12
